### Instead of ingesting ground truth data, we can directly load data from an existing user's table and use them as ground truth.

The only requirement is that users need to supply with `user_schema_mapping` to specify the column names for `query`, `expected_response`, etc.

For ground truth schema, refer to `src/core/trulens/core/schema/groundtruth.py`

In [ ]:
user_specified_schema_name = "GT_QUERY_SET_TEST"

In [ ]:
import os

from trulens.connectors.snowflake import SnowflakeConnector
from trulens.core import TruSession

connection_params = {
    "account": os.environ["SNOWFLAKE_ACCOUNT"],
    "user": os.environ["SNOWFLAKE_USER"],
    "password": os.environ["SNOWFLAKE_USER_PASSWORD"],
    "role": os.environ.get("SNOWFLAKE_ROLE", "ENGINEER"),
    "database": os.environ.get("SNOWFLAKE_DATABASE"),
    "schema": user_specified_schema_name,
    "warehouse": os.environ.get("SNOWFLAKE_WAREHOUSE"),
}
connector = SnowflakeConnector(**connection_params)
session = TruSession(connector=connector)

In [ ]:
session.migrate_database()

In [ ]:
user_schema_mapping = {
    "query": "query",
    "expected_response": "answer_gt",
    "query_id": "query_id",
}


user_df = session.get_virtual_ground_truth(
    "USER_QUERY_SET",
    user_schema_mapping,
    user_schema_name=user_specified_schema_name,
)

In [ ]:
from openai import OpenAI
from trulens.apps.custom import instrument

oai_client = OpenAI()


class APP:
    @instrument
    def completion(self, prompt):
        completion = (
            oai_client.chat.completions.create(
                model="gpt-4o-mini",
                temperature=0,
                messages=[
                    {
                        "role": "user",
                        "content": f"Please answer the question: {prompt}",
                    }
                ],
            )
            .choices[0]
            .message.content
        )
        return completion


llm_app = APP()

In [ ]:
from trulens.core import Feedback
from trulens.feedback import GroundTruthAgreement
from trulens.providers.openai import OpenAI as fOpenAI

f_groundtruth = Feedback(
    GroundTruthAgreement(user_df, provider=fOpenAI()).agreement_measure,
    name="Ground Truth Semantic Agreement",
).on_input_output()

In [ ]:
# add trulens as a context manager for llm_app
from trulens.apps.custom import TruCustomApp

tru_app = TruCustomApp(
    llm_app, app_name="LLM App", app_version="v1", feedbacks=[f_groundtruth]
)

In [ ]:
# Instrumented query engine can operate as a context manager:
with tru_app as recording:
    for _, query in user_df.iterrows():
        resp = llm_app.completion(query)
        print(resp)

In [ ]:
session.get_leaderboard()